<a href="https://colab.research.google.com/github/harperd17/energy_volatility_prediction/blob/main/notebooks/Data%20Ingestion/Futures_Data_Ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Goal: Ingest Futures Data using yahoo finance

In [ ]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/Shareddrives/Data606_Energy/data/futures

/content/drive/Shareddrives/Data606_Energy/data/futures


In [ ]:
futures_symbols = {'BZ=F':'Brent Crude Oil','NG=F':'Natural Gas','CL=F':'Light Sweet Crude Oil','HO=F':'Heating Oil'}

In [ ]:
def get_futures_summaries(data,column_addition_string):
  # first break down the dataset by year
  data['year'] = data.index.year
  data['month'] = data.index.month
  unique_years = list(range(2000,2022))#data['year'].unique()
  mean_movement = []
  std_movement = []
  cv_movement = []
  mean_volume = []
  std_volume = []
  cv_volume = []
  # mean_open_interest = []
  # std_open_interest = []
  dates = []
  for year in unique_years:
    for month in range(1,13):
      dates.append(datetime(year=year,month=month,day=1))
      data_year_month = data[(data['year']==year)&(data['month']==month)]
      # get the average price movement within a day
      mean_movement.append(((data_year_month['High']-data_year_month['Low'])/data_year_month['Open']).mean())
      std_movement.append(((data_year_month['High']-data_year_month['Low'])/data_year_month['Open']).std())
      cv_movement.append(data_year_month['Close'].std()/data_year_month['Close'].mean())
      mean_volume.append(data_year_month['Volume'].mean())
      std_volume.append(data_year_month['Volume'].std())
      cv_volume.append(data_year_month['Volume'].std()/data_year_month['Volume'].mean())
  return {'Date':dates,'Mean Movement'+column_addition_string:mean_movement,'Std Movement'+column_addition_string:std_movement,'CV Movement'+column_addition_string:cv_movement,
          'Mean Volume'+column_addition_string:mean_volume,'Std Volume'+column_addition_string:std_volume,'CV Volume'+column_addition_string:cv_volume}#, 'Mean Open Interest'+column_addition_string:mean_open_interest,'Std Open Interest'+column_addition_string:std_open_interest}

In [ ]:
symbol_summaries = []
symbols_daily_data = []
symbols_weekly_data = []
for symbol in futures_symbols:
  # get summaries at the daily level
  daily_data = yf.download(tickers=symbol,start = '2000-01-01',end='2021-01-01',interval = '1d')
  symbols_daily_data.append(daily_data)
  daily_summaries = pd.DataFrame(get_futures_summaries(daily_data.copy(),' Daily '+symbol))
  # get summaries at the weekly level
  weekly_data = yf.download(tickers=symbol,start = '2000-01-01',end='2021-01-01',interval = '1wk')
  symbols_weekly_data.append(weekly_data)
  weekly_summaries = pd.DataFrame(get_futures_summaries(weekly_data.copy(),' Weekly '+symbol))
  # # get the summaries at the monthly level
  # monthly_data = yf.download(tickers=symbol,start = '2000-01-01',end='2021-01-01',interval = '1mo')
  # monthly_summaries = pd.DataFrame(get_futures_summaries(monthly_data.copy(),' Monthly '+symbol))
  # combine it all into one dataframe
  all_time_frames = pd.merge(daily_summaries,weekly_summaries,on='Date',how='outer')
  #all_time_frames = pd.merge(all_time_frames,monthly_summaries,on='Year')
  symbol_summaries.append(all_time_frames)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
# now go through the dataframes and create one large dataframe
try:
  del grand_data_frame
except:
  ...
for symbol_summary in symbol_summaries:
  # see if the dataframe has been started
  try:
    grand_data_frame
    # if it exists, merge the current dataframe to it
    grand_data_frame = pd.merge(grand_data_frame, symbol_summary,on='Date',how='outer')
  except:
    # if it doesn't exist, create it!
    grand_data_frame = symbol_summary.copy()

In [ ]:
#grand_data_frame.to_csv('futures_data.csv',index=False)

In [ ]:
# here I am creating a standardized data set that it standardized based on rolling means and rolling standard deviations
# the reason for doing this is that for many of these contracts, the average volume increases as time goes on. However, we want to
# know about the impact of volume that is above or below the normal ammount
data_standardized = grand_data_frame.copy()
period = 12#period = 21
for col in data_standardized.columns:
  # standardize the columns which have monthly average to be (avg - rollinging avg)/(rolling std)
  if 'Mean' in col:
    # how the mean of each month/year compare to the means of the last "period" number of months in comparison to the last "period" of months' standard deviations in the underlying column (either price movement or volume)
    # create a column that is the average for that (month - moving average of monthly averages)/(moving average of monthly standard deviations)
    data_standardized[col+' Standardized'] = (data_standardized[col] - data_standardized[col].rolling(period).mean())/data_standardized[col.replace('Mean','Std')].rolling(period).mean()
  # standardize the columns which have monthly standard deviation to be (std - rolling mean of std)/(rolling std)
  if 'Std' in col:
    # how the standard deviation compare to the previous "period" number of months - is the variation in the underlying data (either price movement or volume) higher or lower than expected
    data_standardized[col+' Standardized'] = (data_standardized[col]-data_standardized[col].rolling(period).mean())/data_standardized[col].rolling(period).std()

In [ ]:
# I also want to include some variables for the coefficient of variances
for ticker in ['BZ=F','NG=F','CL=F','HO=F']:
  # another set where the CV is for the rolling values from the past  "period" number of months
  data_standardized['CV Movement Rolling '+ticker] = data_standardized['Std Movement Daily '+ticker].rolling(period).std()/data_standardized['Mean Movement Daily '+ticker].rolling(period).mean()
  data_standardized['CV Volume Rolling '+ticker] = data_standardized['Std Volume Daily '+ticker].rolling(period).std()/data_standardized['Mean Volume Daily '+ticker].rolling(period).mean()

In [ ]:
data_standardized.index = data_standardized['Date']
data_standardized['Year'] = data_standardized.index.year
data_standardized = data_standardized.reset_index(drop=True)

In [ ]:
data_standardized[~data_standardized['Mean Volume Weekly HO=F Standardized'].isnull()]

,Date,Mean Movement Daily BZ=F,Std Movement Daily BZ=F,CV Movement Daily BZ=F,Mean Volume Daily BZ=F,Std Volume Daily BZ=F,CV Volume Daily BZ=F,Mean Movement Weekly BZ=F,Std Movement Weekly BZ=F,CV Movement Weekly BZ=F,Mean Volume Weekly BZ=F,Std Volume Weekly BZ=F,CV Volume Weekly BZ=F,Mean Movement Daily NG=F,Std Movement Daily NG=F,CV Movement Daily NG=F,Mean Volume Daily NG=F,Std Volume Daily NG=F,CV Volume Daily NG=F,Mean Movement Weekly NG=F,Std Movement Weekly NG=F,CV Movement Weekly NG=F,Mean Volume Weekly NG=F,Std Volume Weekly NG=F,CV Volume Weekly NG=F,Mean Movement Daily CL=F,Std Movement Daily CL=F,CV Movement Daily CL=F,Mean Volume Daily CL=F,Std Volume Daily CL=F,CV Volume Daily CL=F,Mean Movement Weekly CL=F,Std Movement Weekly CL=F,CV Movement Weekly CL=F,Mean Volume Weekly CL=F,Std Volume Weekly CL=F,CV Volume Weekly CL=F,Mean Movement Daily HO=F,Std Movement Daily HO=F,CV Movement Daily HO=F,...,Std Movement Daily BZ=F Standardized,Mean Volume Daily BZ=F Standardized,Std Volume Daily BZ=F Standardized,Mean Movement Weekly BZ=F Standardized,Std Movement Weekly BZ=F Standardized,Mean Volume Weekly BZ=F Standardized,Std Volume Weekly BZ=F Standardized,Mean Movement Daily NG=F Standardized,Std Movement Daily NG=F Standardized,Mean Volume Daily NG=F Standardized,Std Volume Daily NG=F Standardized,Mean Movement Weekly NG=F Standardized,Std Movement Weekly NG=F Standardized,Mean Volume Weekly NG=F Standardized,Std Volume Weekly NG=F Standardized,Mean Movement Daily CL=F Standardized,Std Movement Daily CL=F Standardized,Mean Volume Daily CL=F Standardized,Std Volume Daily CL=F Standardized,Mean Movement Weekly CL=F Standardized,Std Movement Weekly CL=F Standardized,Mean Volume Weekly CL=F Standardized,Std Volume Weekly CL=F Standardized,Mean Movement Daily HO=F Standardized,Std Movement Daily HO=F Standardized,Mean Volume Daily HO=F Standardized,Std Volume Daily HO=F Standardized,Mean Movement Weekly HO=F Standardized,Std Movement Weekly HO=F Standardized,Mean Volume Weekly HO=F Standardized,Std Volume Weekly HO=F Standardized,CV Movement Rolling BZ=F,CV Volume Rolling BZ=F,CV Movement Rolling NG=F,CV Volume Rolling NG=F,CV Movement Rolling CL=F,CV Volume Rolling CL=F,CV Movement Rolling HO=F,CV Volume Rolling HO=F,Year
19,2001-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.059086,0.037053,0.110806,28764.173913,13327.493090,0.463337,0.160392,0.045092,0.140393,141053.50,33481.046484,0.237364,0.022945,0.007454,0.019192,63650.608696,20279.500314,0.318607,0.052301,0.014410,0.022641,313907.75,52410.533937,0.166962,0.024485,0.010145,0.023905,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.390747,1.375126,-0.067545,1.186496,1.123163,1.019606,-0.042027,0.155795,-0.674466,-1.757786,-0.149666,0.178118,-0.942170,-0.732343,-0.106627,-0.244433,-0.586980,-0.767849,-0.799095,-0.264297,-0.472114,-0.443604,-1.029627,-0.299533,NaN,NaN,0.202735,0.101312,0.083037,0.046731,0.149257,0.063979,2001
20,2001-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049034,0.014263,0.097745,11657.562500,12657.299370,1.085759,0.173100,0.066637,0.090462,46630.25,35713.108820,0.765879,0.041976,0.028461,0.099033,57879.875000,39237.013019,0.677904,0.105206,0.045475,0.098937,231519.50,154561.701756,0.667597,0.045281,0.033923,0.098301,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.099474,-0.933936,-1.602024,0.950303,1.101479,2.132695,-2.764385,0.272378,0.807094,2.850148,-0.314214,2.779845,1.011855,1.114661,-1.116325,2.019221,0.738368,2.426444,-0.931222,2.355037,0.822834,0.611981,-1.880284,1.414147,NaN,NaN,0.190543,0.109559,0.171665,0.099690,0.223412,0.091814,2001
21,2001-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.073091,0.035400,0.128645,34235.173913,9063.167797,0.264733,0.156749,0.018203,0.154624,165166.60,32849.677796,0.198888,0.034608,0.014579,0.023623,59992.565217,13940.306058,0.232367,0.083796,0.027432,0.043084,294679.00,24704.667403,0.083836,0.029945,0.013052,0.025175,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.765425,1.036153,0.719529,-0.173972,0.609589,-1.2

In [ ]:
data_standardized.to_csv('all_futures_data.csv')

# A quick look at the raw data

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
df = symbols_daily_data[2]
fig = fig.add_trace(go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                ),secondary_y=True)
fig.add_trace(go.Bar(x=df.index, y=df['Volume']),
               secondary_y=False)

fig.show()